In [1]:
'''
Gavin Zhou
1/9/2022

This is a python minesweeper game! Right click to mark squares as bombs, and left click to clear squares.
Win the game by clearing all squares without bombs. Game over if you detonate a bomb!
This program uses tkinter and a 2D array to form the game frame. You can customize the height, width, and
number of bombs by changing the myGrid variable at the bottom. Enjoy!
'''
from tkinter import *
from PIL import Image, ImageTk
import random
import tkinter as tk
import datetime

class Grid(Frame):
    def __init__(self, root, height, width, numBombs):
        Frame.__init__(self, root)
        self.root = root
        self.height = height
        self.width = width
        self.numBombs = numBombs
        self.buttons = [[None for _ in range(width)] for _ in range(height)] #buttons
        self.values = [[None for _ in range(width)] for _ in range(height)] # save the number
        self.watch = ""
        self.flagCounter = numBombs
        self.sunkenCounter = 0
        self.begin = datetime.datetime.now()
        ## create grid of buttons
        for j in range(self.width):
            for i in range(self.height):
                self.buttons[i][j] = Button(self.root, height = 2, width = 4, bg = 'green')
                self.buttons[i][j].config(command = lambda x = i, y = j: self.onClick(x, y))
                self.buttons[i][j].bind('<Button-3>', lambda event, x = i, y = j: self.onRightClick(x, y))
                self.buttons[i][j].grid(row = i, column = j, columnspan = 1)
        ## display user information, like flags and time
        self.labelFlagsRemaining = Label(self.root, text = str(self.flagCounter), font = ('Helvetica', 18))
        self.labelFlagsRemaining.grid(row = self.height, columnspan = 3, column = self.width // 2 - 1, sticky = S)
        self.watch = Label(self.root, text = "", font = ('Helvetica', 18))
        self.watch.grid(row = self.height, columnspan = 5, column = self.width - 5, sticky = S)

        ## create buttons and bombs
        while True:
            for i in range(self.height):
                for j in range(self.width):
                    self.buttons[i][j].config(text = '', fg = 'grey')
                    self.values[i][j] = ''
            bombLoc = random.sample(range(1, self.width * self.height), numBombs)
            for i in range(self.numBombs):
                bombLocY = bombLoc[i] // self.width
                bombLocX = bombLoc[i] % self.width
                self.buttons[bombLocY][bombLocX].config(text = 'x', fg = 'green')
                self.values[bombLocY][bombLocX] = 'x'

            if self.buttons[0][0]['text'] != 'x' and self.buttons[0][1]['text'] != 'x' and self.buttons[1][0]['text'] != 'x' and self.buttons[1][1]['text'] != 'x':
                break
        ## set each square to display number of bombs around it. If none, square is blank
        for x in range(self.height):
            for y in range(self.width):
                if self.buttons[x][y]['text'] != 'x':
                    neighborNumBombs = 0
                    for i in range(-1, 2):
                        for j in range(-1, 2):                            
                            if x + i >= 0 and x + i < self.height and y + j >= 0 and y + j < self.width:
                                if self.buttons[x + i][y + j]['text'] == 'x':
                                    neighborNumBombs += 1
                    if neighborNumBombs == 0:
                        self.buttons[x][y].config(text = '', fg = 'green')
                        self.values[x][y] = ''
                    else:
                        self.buttons[x][y].config(text = str(neighborNumBombs), fg = 'green')
                        self.values[x][y] = str(neighborNumBombs)
            
    ## set button actions and colors
    def onRightClick(self, x, y):
        if self.buttons[x][y]['relief'] == SUNKEN:
            return    
        
        if self.buttons[x][y]['text'] == 'X':
            self.buttons[x][y].config(text = self.values[x][y], fg = 'green')
            self.flagCounter += 1
        elif self.flagCounter <= 0:
            return
        else:
            self.buttons[x][y].config(text = 'X', fg = 'black')
            self.flagCounter -= 1
                        
        self.labelFlagsRemaining.config(text= str(self.flagCounter))
          
    ## clear square and reveal subsequent squares
    def onClick(self, x, y):
        if x < 0 or x >= self.height or y < 0 or y >= self.width or self.buttons[x][y]['relief'] == SUNKEN or self.buttons[x][y]['text'] == '*':
            return
        if self.buttons[x][y]['text'] == 'x':
            print('game over')
            newWindow = Toplevel(root)
            newWindow.geometry("200x50") 
            Label(newWindow,text ="Game Over! Try again next time :(").pack() ## display message for a loss
            self.end = datetime.datetime.now()
            self.watch.config(text = str(self.end - self.begin))
            for i in range(self.height):
                for j in range(self.width):
                    self.buttons[i][j].config(text = self.values[i][j])
                    self.buttons[i][j].config(state = DISABLED, relief = RAISED, bg = 'red', fg = 'black')
                    
        ## reveal touching clear squares
        elif self.buttons[x][y]['text'] == '':
            self.buttons[x][y].config(state = DISABLED, relief = SUNKEN, bg = 'white', fg = 'black')
            self.sunkenCounter += 1
            self.isWon()
            for i in range(-1, 2):
                for j in range(-1, 2):
                    self.onClick(x + i, y + j)
        else: # some number
            self.buttons[x][y].config(state = DISABLED, relief = SUNKEN, bg = 'white', fg = 'black')
            self.sunkenCounter += 1
            self.isWon()

    def isWon(self): # check if all available squares are cleared, if so then game is won. Shows time high score
        if self.sunkenCounter + self.numBombs == self.width * self.height:
            self.end = datetime.datetime.now()
            self.watch.config(text = str(self.end - self.begin))
            newWindow = Toplevel(root)
            newWindow.geometry("300x50") 
            Label(newWindow,text = "Yay! you won! Your high score was " + str(self.end - self.begin)).pack()
            for i in range(self.height): # disable all buttons
                for j in range(self.width):
                    self.buttons[i][j].config(text = self.values[i][j])
                    self.buttons[i][j].config(state = DISABLED, bg = 'green', fg = 'black')

root = Tk()
myGrid = Grid(root, 15, 15, 15)

root.mainloop()

game over
